## Predict rainfall and drought using long-term data from Weather.com

____
____

#### Presentors:
- Amirabbas RezaSoltani
- Amin Ariafar
</br> </br>
#### Professor: Dr. Abdollah Safari

#### University of Tehran

#### Summer 2024

____
____

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gamma
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor


First, we import the dataset.

In [2]:
df = pd.read_csv('Tehran 2009-2019.csv', encoding='unicode_escape', parse_dates=['Year'])
df = df.rename(columns={'Year': 'data'})

In [3]:
df['year_month'] = df['data'].dt.to_period('M')
df['year'] = df['data'].dt.to_period('Y')

In [4]:
df.head()

,data,Weather,Max,Min,Wind,Rain,Humidity,Cloud,Pressure,year_month,year
0,2009-01-01,Partly cloudy,5 °c,-5 °c,4 km/hSSW,0.0 mm,62%,26%,1022 mb,2009-01,2009
1,2009-01-02,Sunny,4 °c,0 °c,5 km/hS,3.4 mm,74%,32%,1014 mb,2009-01,2009
2,2009-01-03,Sunny,4 °c,-3 °c,12 km/hW,0.0 mm,60%,6%,1016 mb,2009-01,2009
3,2009-01-04,Sunny,3 °c,-7 °c,9 km/hWNW,0.0 mm,35%,1%,1022 mb,2009-01,2009
4,2009-01-05,Sunny,4 °c,-7 °c,3 km/hW,0.0 mm,23%,14%,1023 mb,2009-01,2009


We need to do some data cleaning so that our dataset can be fed to our learning algorithms.

In [5]:
df['Max'] = df['Max'].str.slice(0, -3).astype('int')
df['Min'] = df['Min'].str.slice(0, -3).astype('int')
df['Rain'] = df['Rain'].str.slice(0, -3).astype('float')
df['Humidity'] = df['Humidity'].str.slice(0, -1).astype('int')
df['Cloud'] = df['Cloud'].str.slice(0, -1).astype('int')
df['Pressure'] = df['Pressure'].str.slice(0, -3).astype('int')


wind = df['Wind'].str.split(' ', expand = True)
df['Wind'] = wind[0].astype(int)
one_hot_encoded = pd.get_dummies(wind[1], prefix='wind_unit')
df = pd.concat([df, one_hot_encoded], axis = 1)


# dictionary for replacing weather labels with numeric values (with keeping the logical order of labels)
weather_dict = {
    'Sunny': 1,
    'Partly cloudy': 2,
    'Cloudy': 3,
    'Overcast': 4,
    'Patchy light drizzle': 5,
    'Light drizzle': 6,
    'Patchy rain possible': 7,
    'Light rain': 8,
    'Patchy light rain': 9,
    'Light rain shower': 10,
    'Moderate rain at times': 11,
    'Moderate rain': 12,
    'Heavy rain at times': 13,
    'Heavy rain': 14,
    'Moderate or heavy rain shower': 15,
    'Patchy light rain with thunder': 16,
    'Moderate or heavy rain with thunder': 17,
    'Thundery outbreaks possible': 18,
    'Patchy snow possible': 19,
    'Patchy light snow': 20,
    'Light freezing rain': 21,
    'Light sleet': 22,
    'Patchy sleet possible': 23,
    'Moderate snow': 24,
    'Heavy snow': 25,
    'Patchy moderate snow': 26,
    'Patchy heavy snow': 27,
    'Moderate or heavy snow showers': 28
}
df['Weather'] = df["Weather"].replace(to_replace = weather_dict)


display(df.head())
print(df.info())

,data,Weather,Max,Min,Wind,Rain,Humidity,Cloud,Pressure,year_month,...,wind_unit_km/hNNW,wind_unit_km/hNW,wind_unit_km/hS,wind_unit_km/hSE,wind_unit_km/hSSE,wind_unit_km/hSSW,wind_unit_km/hSW,wind_unit_km/hW,wind_unit_km/hWNW,wind_unit_km/hWSW
0,2009-01-01,2,5,-5,4,0.0,62,26,1022,2009-01,...,0,0,0,0,0,1,0,0,0,0
1,2009-01-02,1,4,0,5,3.4,74,32,1014,2009-01,...,0,0,1,0,0,0,0,0,0,0
2,2009-01-03,1,4,-3,12,0.0,60,6,1016,2009-01,...,0,0,0,0,0,0,0,1,0,0
3,2009-01-04,1,3,-7,9,0.0,35,1,1022,2009-01,...,0,0,0,0,0,0,0,0,1,0
4,2009-01-05,1,4,-7,3,0.0,23,14,1023,2009-01,...,0,0,0,0,0,0,0,1,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4333 entries, 0 to 4332
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               4333 non-null   datetime64[ns]
 1   Weather            4333 non-null   int64         
 2   Max                4333 non-null   int64         
 3   Min                4333 non-null   int64         
 4   Wind               4333 non-null   int64         
 5   Rain               4333 non-null   float64       
 6   Humidity           4333 non-null   int64         
 7   Cloud              4333 non-null   int64         
 8   Pressure           4333 non-null   int64         
 9   year_month         4333 non-null   period[M]     
 10  year               4333 non-null   period[A-DEC] 
 11  wind_unit_km/hE    4333 non-null   uint8         
 12  wind_unit_km/hENE  4333 non-null   uint8         
 13  wind_unit_km/hESE  4333 non-null   uint8         
 14  wind_uni

# Task1

It is required to convert data to monthly format and aggregate raining value in a month 

In [6]:
monthly_rain = df.groupby('year_month')['Rain'].sum().reset_index()

In [7]:
monthly_rain.head()

,year_month,Rain
0,2009-01,10.0
1,2009-02,9.9
2,2009-03,3.2
3,2009-04,33.1
4,2009-05,13.8


Seperate days based on their SPI.

In [8]:
def seperate(TRESH_DROUGHT, TRESH_TRIBBLE, data = monthly_rain): 
   rain = data['Rain'].values
   shape, loc, scale = gamma.fit(rain)
   gam = gamma.cdf(rain, shape, loc, scale)

   # transform the Gamma CDF to the standard normal distribution (SPI)
   data['spi'] = norm.ppf(gam)

   # denote drought as 0 and wetness as 1
   conditions = [(data['spi'] <= TRESH_TRIBBLE) & (data['spi'] >= TRESH_DROUGHT), 
               (data['spi'] > TRESH_TRIBBLE),
               (data['spi'] < TRESH_DROUGHT)]
   tasks = [0, -1, 1] #0 is mediocre, 1 id drought and -1 is tribble
   data['drought'] = np.select(conditions, tasks)
   print('Splited Data:')
   display(data.head())
   print('Value Count on Dataset')
   display(data.drought.value_counts())

Train test split

In [9]:
def split(test_size, data = monthly_rain, col = 'drought'):
   test = data[-test_size:][[col]].reset_index(drop=True).values
   train = data[:data.shape[0]-test_size][[col]].reset_index(drop=True).values
   return train, test

Train and test descion tree classifier

In [10]:
def train_test_class(size, X_train, X_test, y_train, y_test, max_d = 5):
    mean_scores = []
    max_depth_values = []
    for max_depth in range(1, max_d+1):
        max_depth_values.append(max_depth)
        clf = DecisionTreeClassifier(max_depth=max_depth, random_state=20)
        scores = cross_val_score(clf, X_train, y_train, cv=6)
        mean_scores.append(np.mean(scores))
    best_max_depth_idx = np.argmax(mean_scores)
    best_max_depth = max_depth_values[best_max_depth_idx]
    clf = DecisionTreeClassifier(max_depth=best_max_depth)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for window size of {size}: {accuracy:.2f}")
    return clf, accuracy

def windowing(size, test, train):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(test.shape[0] - size):
        x_test.append(test[i:i+size].reshape(1,-1)[0])
        y_test.append(test[i+size][0])
    for i in range(train.shape[0] - size):
        x_train.append(train[i:i+size].reshape(1,-1)[0])
        y_train.append(train[i+size][0])
    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

test_size = 15 #define test size to split
input_size = range(1, 6) #max_depth tree values, try to choose bast value using cross validatoin

In [11]:
def set_maker(size, values):
    res = []
    def backtrack(path = []):
        if len(path) == size:
            res.append(path.copy())
            return
        else:
            for i in values:
                backtrack(path + [i])
    backtrack()
    return res

Compute the SPI factor in monthly scale
2 approcches are tested here. First is 3 level output (-1, 0, 1) based on SPI. -1 is for terrible, 0 is mediocre and 1 is drought.
Other approch is 2 level output (-1, 1) based on SPI. -1 is for terrible and 1 is drought.

## 2 level approach

In [12]:
seperate(0, 0)
train, test = split(test_size)
models = []
accs = []
for i in input_size:
    x_train, y_train, x_test, y_test = windowing(i, test, train)
    mod, acc = train_test_class(i, x_train, x_test, y_train, y_test)
    models.append(mod)
    accs.append(acc)
best_model = models[np.argmax(accs)]
print(f'Best model in on data with window size of {np.argmax(accs) + 1} with accuracy of {np.max(accs) * 100:0.2f}%')

Splited Data:


,year_month,Rain,spi,drought
0,2009-01,10.0,-0.237345,1
1,2009-02,9.9,-0.244852,1
2,2009-03,3.2,-0.973704,1
3,2009-04,33.1,0.847003,-1
4,2009-05,13.8,0.015265,-1


Value Count on Dataset


-1    74
 1    69
Name: drought, dtype: int64

Accuracy for window size of 1: 0.79
Accuracy for window size of 2: 0.85
Accuracy for window size of 3: 0.83
Accuracy for window size of 4: 0.82
Accuracy for window size of 5: 0.80
Best model in on data with window size of 2 with accuracy of 84.62%


In [13]:
state_out = [-1, 1]
state_inp = set_maker(np.argmax(accs) + 1, state_out)
prob = pd.DataFrame(index=[str(inx) for inx in state_inp], columns=state_out)
for i in state_inp:
    p = best_model.predict_proba([i])
    prob.loc[str(i), :] = p
print('Transition Matrix:')
display(prob)

Transition Matrix:


,-1,1
"[-1, -1]",0.671875,0.328125
"[-1, 1]",0.33871,0.66129
"[1, -1]",0.671875,0.328125
"[1, 1]",0.33871,0.66129


## 3 levels approach

In [14]:
seperate(-0.49, 0.49)
train, test = split(test_size)
models = []
accs = []
for i in input_size:
    x_train, y_train, x_test, y_test = windowing(i, test, train)
    mod, acc = train_test_class(i, x_train, x_test, y_train, y_test)
    models.append(mod)
    accs.append(acc)
best_model = models[np.argmax(accs)]
print(f'Best model in on data with window size of {np.argmax(accs) + 1} with accuracy of {np.max(accs) * 100:0.2f}%')

Splited Data:


,year_month,Rain,spi,drought
0,2009-01,10.0,-0.237345,0
1,2009-02,9.9,-0.244852,0
2,2009-03,3.2,-0.973704,1
3,2009-04,33.1,0.847003,-1
4,2009-05,13.8,0.015265,0


Value Count on Dataset


 0    59
-1    44
 1    40
Name: drought, dtype: int64

Accuracy for window size of 1: 0.64
Accuracy for window size of 2: 0.69
Accuracy for window size of 3: 0.67
Accuracy for window size of 4: 0.64
Accuracy for window size of 5: 0.60
Best model in on data with window size of 2 with accuracy of 69.23%


In [15]:
state_out = [-1, 0, 1]
state_inp = set_maker(np.argmax(accs) + 1, state_out)
prob = pd.DataFrame(index=[str(inx) for inx in state_inp], columns=state_out)
for i in state_inp:
    p = best_model.predict_proba([i])
    prob.loc[str(i), :] = p
print('Transition Matrix:')
display(prob)

Transition Matrix:


,-1,0,1
"[-1, -1]",0.470588,0.470588,0.058824
"[-1, 0]",0.195652,0.434783,0.369565
"[-1, 1]",0.195652,0.434783,0.369565
"[0, -1]",0.470588,0.470588,0.058824
"[0, 0]",0.195652,0.434783,0.369565
"[0, 1]",0.195652,0.434783,0.369565
"[1, -1]",0.470588,0.470588,0.058824
"[1, 0]",0.195652,0.434783,0.369565
"[1, 1]",0.195652,0.434783,0.369565


# Task2

Assume the chain is discrete-time gaussian markov chain. So the raining value of each day is only related to prevoius n days. Also Raining value of each day (and linear combination of rainig of every indivitual day) is from normal distributaion.

In [16]:
daily_rain = df.loc[:, ['Rain']]
daily_rain.head()

,Rain
0,0.0
1,3.4
2,0.0
3,0.0
4,0.0


In [17]:
def train_test_reg(size, X_train, X_test, y_train, y_test, max_d = 10):
    mean_scores = []
    max_depth_values = []
    for max_depth in range(1, max_d+1):
        max_depth_values.append(max_depth)
        clf = RandomForestRegressor(n_estimators = 100, max_depth=max_depth, random_state=20, max_features = None)
        scores = cross_val_score(clf, X_train, y_train, cv=6, scoring='neg_mean_squared_error')
        mean_scores.append(np.mean(scores))
    best_max_depth_idx = np.argmax(mean_scores)
    best_max_depth = max_depth_values[best_max_depth_idx]
    clf = RandomForestRegressor(n_estimators = 100, max_depth=best_max_depth, random_state=20, max_features = None)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE for window size of {size}: {mse:.2f}")
    return clf, mse

In [18]:
test_size = int(train.shape[0] * 0.15)
train, test = split(test_size, data = daily_rain, col = 'Rain')
models = []
accs = []
for i in input_size:
    x_train, y_train, x_test, y_test = windowing(i, test, train)
    mod, acc = train_test_reg(i, x_train, x_test, y_train, y_test)
    models.append(mod)
    accs.append(acc)
best_model = models[np.argmin(accs)]
print(f'Best model in on data with window size of {np.argmin(accs) + 1} with MSE of {np.min(accs):0.2f}')

MSE for window size of 1: 24.39
MSE for window size of 2: 28.10
MSE for window size of 3: 30.56
MSE for window size of 4: 30.47
MSE for window size of 5: 34.09
Best model in on data with window size of 1 with MSE of 24.39


In [19]:
def calculate_normal_dist(model, inp):
    all_tree_predictions = np.array([tree.predict(inp) for tree in model.estimators_])
    mean_predictions = np.mean(all_tree_predictions, axis=0)
    variance_predictions = np.var(all_tree_predictions, axis=0)
    return mean_predictions, variance_predictions

#some examples
for i in range(10):
    inp = []
    for j in range(1, np.argmin(accs) + 2):
        inp.append(np.random.random() * 10)
    inp = [inp]
    mean, var = calculate_normal_dist(best_model, inp)
    print(f'Output for input {inp[0]} has normal distribution with mean {mean[0]:0.2f} and variance {var[0]:0.2f}.')

Output for input [6.46065676795856] has normal distribution with mean 3.15 and variance 7.93.
Output for input [7.027461310884899] has normal distribution with mean 4.65 and variance 39.69.
Output for input [9.462407814254528] has normal distribution with mean 3.21 and variance 1.48.
Output for input [9.432941740441693] has normal distribution with mean 3.21 and variance 1.48.
Output for input [3.1831074852788053] has normal distribution with mean 1.88 and variance 0.26.
Output for input [1.5558748607326056] has normal distribution with mean 1.78 and variance 0.24.
Output for input [0.7739171694174019] has normal distribution with mean 0.56 and variance 0.02.
Output for input [8.146697460010499] has normal distribution with mean 3.21 and variance 1.48.
Output for input [5.197417321290018] has normal distribution with mean 1.92 and variance 0.32.
Output for input [9.600016090364216] has normal distribution with mean 3.21 and variance 1.48.


# Task3

In [20]:
df.head()

,data,Weather,Max,Min,Wind,Rain,Humidity,Cloud,Pressure,year_month,...,wind_unit_km/hNNW,wind_unit_km/hNW,wind_unit_km/hS,wind_unit_km/hSE,wind_unit_km/hSSE,wind_unit_km/hSSW,wind_unit_km/hSW,wind_unit_km/hW,wind_unit_km/hWNW,wind_unit_km/hWSW
0,2009-01-01,2,5,-5,4,0.0,62,26,1022,2009-01,...,0,0,0,0,0,1,0,0,0,0
1,2009-01-02,1,4,0,5,3.4,74,32,1014,2009-01,...,0,0,1,0,0,0,0,0,0,0
2,2009-01-03,1,4,-3,12,0.0,60,6,1016,2009-01,...,0,0,0,0,0,0,0,1,0,0
3,2009-01-04,1,3,-7,9,0.0,35,1,1022,2009-01,...,0,0,0,0,0,0,0,0,1,0
4,2009-01-05,1,4,-7,3,0.0,23,14,1023,2009-01,...,0,0,0,0,0,0,0,1,0,0
